In [4]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json
from decouple import config
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob

In [5]:
consumer_key =  config('consumer_key')
consumer_secret = config('consumer_secret')
access_token = config('access_token')
access_secret = config('access_secret')

In [7]:
class TweetsListener(StreamListener):
    # initialized the constructor
    def __init__(self, csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:
            # read the Twitter data which comes as a JSON format
            msg = json.loads(data)
            # the 'text' in the JSON file contains the actual tweet.
            print(msg['text'].encode('utf-8'))

            # the actual tweet data is sent to the client socket
            self.client_socket.send(msg['text'].encode('utf-8'))
            return True

        except BaseException as e:
            # Error handling
            print("Ahh! Look what is wrong : %s" % str(e))
            return True

    def on_error(self, status):
        print(status)
        return True


def sendData(c_socket):
    # authentication
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # twitter_stream will get the actual live tweet data
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    # filter the tweet feeds related to "corona"
    twitter_stream.filter(track=['corona'])
    # in case you want to pass multiple criteria
    # twitter_stream.filter(track=['DataScience','python','Iot'])


# create a socket object
s = socket.socket()

# Get local machine name : host and port
host = "127.0.0.1"
port = 3334

# Bind to the port
s.bind((host, port))
print("Listening on port: %s" % str(port))
# Wait and Establish the connection with client.
s.listen(5)
c, addr = s.accept()
print("Received request from: " + str(addr))
# Keep the stream data available
sendData(c)

rror 10053] An established connection was aborted by the software in your host machine
b'RT @JAlbajari: #Corona crisis in #India is a humanitarian catastrophe. It is chilling!More than 350k infections every day and 1death every\xe2\x80\xa6'
Ahh! Look what is wrong : [WinError 10053] An established connection was aborted by the software in your host machine
b"RT @prankya: Let's cooperate with the Corona frontliners and help them to serve their duty in an efficient manner.\n\nTogether we are stronge\xe2\x80\xa6"
Ahh! Look what is wrong : [WinError 10053] An established connection was aborted by the software in your host machine
b'@benfire71 @gerdantes @LTCcovid Bayern, auch Ende Jan., seit Beginn der Pandemie: https://t.co/wF4sAGqwnT'
Ahh! Look what is wrong : [WinError 10053] An established connection was aborted by the software in your host machine
b'@manfredosz @alexnijenhuis @ScoobyPlutodog @GideonvMeijeren En alle overige doden dalen... Aantal griepdoden afgelo\xe2\x80\xa6 https://t

ProtocolError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))